# Test BR00115133-31 Dataset

In [70]:
# Step 1: mkdir filepath to save dataset

# !mkdir -p '../dataset/cpg0001/2020_08_11_Stain3_Yokogawa/'

# Step 2: dowaload 2020_08_11_Stain3_Yokogawa dataset, about 70G
# !aws s3 sync s3://cellpainting-gallery/cpg0001-cellpainting-protocol/source_4/images/2020_08_11_Stain3_Yokogawa/ ./2020_08_11_Stain3_Yokogawa --no-sign-request

In [71]:
# # move noise images such as DC_ to others

# import os
# import shutil

# # 源目录和目标目录
# source_dir = "/data/boom/cpg0001/2020_08_11_Stain3_Yokogawa/images/BR00115134/"
# target_dir = "/data/boom/cpg0001/2020_08_11_Stain3_Yokogawa/images/others/"

# # 确保目标目录存在
# os.makedirs(target_dir, exist_ok=True)

# # 遍历源目录中的所有文件
# for filename in os.listdir(source_dir):
#     file_path = os.path.join(source_dir, filename)
    
#     # 只处理文件（跳过子目录）
#     if os.path.isfile(file_path):
#         # 检查文件名是否不以"BR00"开头
#         if not filename.startswith("BR00"):
#             target_path = os.path.join(target_dir, filename)
            
#             # 移动文件
#             shutil.move(file_path, target_path)
#             # print(f"Moved: {filename} -> {target_dir}")

In [72]:
# # make CSV file

# import os
# import csv

# # 指定包含tif文件的文件夹路径
# folder_path = '/data/boom/cpg0001/2020_08_11_Stain3_Yokogawa/images/BR00115134/'

# # 指定输出CSV文件的路径
# output_csv_path = '/data/boom/cpg0001/2020_08_11_Stain3_Yokogawa/images/BR00115134.csv'

# # 获取文件夹中所有的tif文件
# tif_files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]

# # 获取文件夹中所有的tif文件，并按照文件名排序
# tif_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.tif')])

# # 将文件名写入CSV文件
# with open(output_csv_path, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     for file_name in tif_files:
#         writer.writerow([file_name])

# print(f"All .tif files have been written to {output_csv_path}")

In [73]:
import os
import pandas as pd
import numpy as np
import torch
from skimage.transform import resize
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm
from huggingface_mae import MAEModel
from utils import *

class PDDDataset(torch.utils.data.Dataset):
    def __init__(self, image_path, CSV_path):
        self.image_path = image_path
        self.image_files_CSV = pd.read_csv(CSV_path, header=None)  # 只取文件名列
        self.image_files = self.image_files_CSV[0].tolist()
        self.total_images = len(self.image_files)
        print(len(self.image_files))
        
        # 确保图像数量是5的倍数
        assert self.total_images % 5 == 0, "图像总数必须是5的倍数"
        self.num_groups = self.total_images // 5
        
        # 预处理转换
        self.transform = transforms.Compose([transforms.ToTensor()])
        
    def load_image(self, img_name):
        img_path = os.path.join(self.image_path, img_name)
        image = Image.open(img_path)
        return np.array(image)
    
    def __getitem__(self, idx):
        # 计算5个图像的起始位置
        start_idx = idx * 5
        
        # 加载5个连续的图像
        images = []
        for i in range(5):
            img_name = self.image_files[start_idx + i]
            img_array = self.load_image(img_name)
            images.append(img_array)
        
        # 堆叠图像并调整大小 (5, H, W) -> (5, 448, 448)
        images = np.stack(images, axis=0)
        images = resize(images, (5, 256, 256), anti_aliasing=True)
        
        # 转换为张量 (5, 448, 448) -> (5, 448, 448)
        images = torch.from_numpy(images).float()
        
        return {'image': images}
    
    def __len__(self):
        return self.num_groups


In [74]:
import pandas as pd
image_files = pd.read_csv('/data/boom/cpg0001/2020_08_11_Stain3_Yokogawa/images/BR00115134.csv', header=None)  # 只取文件名列
# image_files

In [75]:
def build_loaders_inference(batch_size):
    print("Building loaders")
    dataset = PDDDataset(
        image_path="/data/boom/cpg0001/2020_08_11_Stain3_Yokogawa/images/BR00115134/",
        CSV_path="/data/boom/cpg0001/2020_08_11_Stain3_Yokogawa/images/BR00115134.csv"
    )
    
    test_loader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=4, 
        pin_memory=True, 
        drop_last=False
    )

    print(f"Total batches: {len(test_loader)}")
    return test_loader

def get_image_embeddings(model_path, model, batch_size):
    test_loader = build_loaders_inference(batch_size)
    model = model.eval().cuda()
    
    print("Finished loading model")
    test_image_embeddings = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader):
            
            # Run prediction
            model.return_channelwise_embeddings = False
            image_embeddings = model.predict(batch["image"].cuda())
            
            # Reshape back if needed (depending on your model output)
            test_image_embeddings.append(image_embeddings)
            # print(image_embeddings.shape)
            # break
    
    return torch.cat(test_image_embeddings)

In [76]:
MODEL_PATH = "recursionpharma/OpenPhenom"
save_path = "output/BR00115134/"

if not os.path.exists(save_path):
    os.makedirs(save_path)

# 加载模型
model = MAEModel.from_pretrained(MODEL_PATH).cuda()
img_embeddings = get_image_embeddings(MODEL_PATH, model, batch_size=200)  # change batch_size to fit your device
features = img_embeddings.cpu().numpy()

np.save(save_path + "OpenPhenom_BR00115134_test" + ".npy", features.T)

/home/bob/boom/PhenoProfiler/Baseline/OpenPhenom/huggingface_mae.py:291: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(modelpath, map_location="cpu")

Building loaders
7680
Total batches: 8
Finished loading model


  0%|          | 0/8 [00:00<?, ?it/s]/home/bob/anaconda3/envs/boom/lib/python3.11/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
100%|██████████| 8/8 [26:16<00:00, 197.03s/it]  


In [77]:
features.shape

(1536, 384)

# Evalution

In [78]:
import numpy as np

save_path = "output/BR00115134/"

# features = np.load(save_path+"OpenPhenom_BR00115134_test.npy").T
# save_path = 'output/BR00115134/'
REG_PARAM = 1e-2
num_features = features.shape[1]

print(features.shape)

(1536, 384)


## Generate csv like below_csv, which have plate well and treatment

In [79]:
# path = '/home/bob/boom/PhenoProfiler/revision/BBBC022/PhenoProfiler/0_noBC_well_level.csv'

# csv = pd.read_csv(path)

# csv.head()

# Make GT of MOA

In [80]:
# read JUMP-MOA_compound_platemap_with_metadata.txt

import pandas as pd

meta = pd.read_csv('/home/bob/boom/PhenoProfiler/output/JUMP-MOA_compound_platemap_with_metadata.txt', sep='\t')  # assuming it's tab-delimited
print(meta.shape)  # meta['moa']
# meta.head()

(384, 11)


In [81]:
meta['pert_type'].unique()

array(['trt', 'control'], dtype=object)

In [82]:
# 整合成 Well Trt Features 的形式

import numpy as np
import pandas as pd

# 加载数据
# save_path = 'output/BR00115134/'
REG_PARAM = 1e-2
# num_features = 672

# features = np.load(save_path + "DeepProfiler_alltrain_25test.npy").T
# meta = pd.read_csv('output/JUMP-MOA_compound_platemap_with_metadata.txt', sep='\t')

# 检查数据形状是否匹配
assert len(meta) == features.shape[0] // 4, "Meta数据行数与特征数据不匹配"

# 特征聚合：每4行取平均
aggregated_features = np.zeros((len(meta), num_features))
for i in range(len(meta)):
    start_idx = i * 4
    end_idx = start_idx + 4
    aggregated_features[i] = features[start_idx:end_idx].mean(axis=0)

# 创建结果DataFrame
result_df = pd.DataFrame({
    'Well': meta['well_position'],
    'Treatment': meta['pert_type'],
    'broad_sample':meta['broad_sample']
})

# 添加聚合后的特征
feature_columns = [i for i in range(num_features)]
result_df[feature_columns] = aggregated_features

# 保存为CSV文件
# output_path = save_path + 'DeepProfiler_Wells_NoBC.csv'
# result_df.to_csv(output_path, index=False)

/tmp/ipykernel_567401/1172509567.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[feature_columns] = aggregated_features
/tmp/ipykernel_567401/1172509567.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[feature_columns] = aggregated_features
/tmp/ipykernel_567401/1172509567.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get 

In [83]:
# result_df.head()

# BC Sphering

In [84]:
import scipy.linalg
import pandas as pd

class WhiteningNormalizer(object):
    def __init__(self, controls, reg_param=1e-6):
        # Whitening transform on population level data
        self.mu = controls.mean()
        self.whitening_transform(controls - self.mu, reg_param, rotate=True)
        # print(self.mu.shape, self.W.shape)
        
    def whitening_transform(self, X, lambda_, rotate=True):
        C = (1/X.shape[0]) * np.dot(X.T, X)
        s, V = scipy.linalg.eigh(C)
        D = np.diag( 1. / np.sqrt(s + lambda_) )
        W = np.dot(V, D)
        if rotate:
            W = np.dot(W, V.T)
        self.W = W

    def normalize(self, X):
        return np.dot(X - self.mu, self.W)

columns2 = [i for i in range(num_features)]  #　(672)
REG_PARAM = 1e-2

wells = result_df
whN = WhiteningNormalizer(wells.loc[wells["Treatment"] == "control", columns2], reg_param=REG_PARAM)

whD = whN.normalize(wells[columns2])

# Save whitened profiles
wells[columns2] = whD

# wells.head()

In [85]:
wells.shape

(384, 387)

# Treatment-level profiles / Mean Aggreagation

In [86]:
import sklearn

# Aggregate profiles
columns1 = ["Well", "broad_sample"]
columns2 = [i for i in range(num_features)]

wells_1 = wells.drop(columns=["Well"])

profiles = wells_1.groupby(["broad_sample", 'Treatment']).mean().reset_index()

# 将 meta 的 broad_sample 和 moa 转为字典
moa_dict = meta.set_index('broad_sample')['moa'].to_dict()

# 用 map 添加 moa 列
profiles['moa'] = profiles['broad_sample'].map(moa_dict)

# profiles = wells_1[["Treatment", "broad_sample"] + columns2]
# # profiles
print(profiles.shape)
# profiles.head()

(90, 387)


In [87]:
# # 6. Similarity matrix
# Compute Cosine Similarities
from sklearn.metrics.pairwise import cosine_similarity

COS = cosine_similarity(profiles[columns2], profiles[columns2])
COS.shape

(90, 90)

In [88]:
# Transform to tidy format
df = pd.DataFrame(data=COS, index=list(profiles.broad_sample), columns=list(profiles.broad_sample))
#　将行索引重置为默认整数索引，并将原来的行索引 broad_sample 转换为一列，命名为 index。所以，variable　表示原来的broad_sample名。
df = df.reset_index().melt(id_vars=["index"])
# df # 其中每一行都表示 预测的Treatment和 GT 之间的概率。

# Annotate rows
df2 = pd.merge(
    df, 
    profiles[["broad_sample", "moa"]],  # 为了加 Metadata_moa.x 列数据，先用 broad_sample 建立对应关系，然后删除。
    how="left", 
    left_on="index", # <=== Rows
    right_on="broad_sample"
).drop("broad_sample",axis=1)

# Annotate columns
#　index　和 variable 是一个东西，都表示 Treatment，但是 index　对应 Metadata_moa.x_x	，variable对应Metadata_moa.x_y
df2 = pd.merge(
    df2, profiles[["broad_sample", "moa"]],
    how="left", 
    left_on="variable", # <=== Columns
    right_on="broad_sample"
).drop("broad_sample",axis=1)

df2.head()

,index,variable,value,moa_x,moa_y
0,BRD-A12994259-001-02-1,BRD-A12994259-001-02-1,1.000000,tumor necrosis factor production inhibitor,tumor necrosis factor production inhibitor
1,BRD-A22769835-300-05-7,BRD-A12994259-001-02-1,0.165413,antihistamine,tumor necrosis factor production inhibitor
2,BRD-A53576514-048-14-3,BRD-A12994259-001-02-1,0.299385,acetylcholine receptor antagonist,tumor necrosis factor production inhibitor
3,BRD-A78210457-001-01-5,BRD-A12994259-001-02-1,0.287150,MDM inhibitor,tumor necrosis factor production inhibitor
4,BRD-A87435144-001-01-6,BRD-A12994259-001-02-1,-0.102513,pyruvate dehydrogenase kinase inhibitor,tumor necrosis factor production inhibitor


In [89]:
df2.shape

(8100, 5)

In [90]:
# Rename columns and save
df2.columns = ["Var1", "Var2", "value", "Metadata_moa.x", "Metadata_moa.y"]

# # MOA Evaluation using enrichment analysis

SIM_MATRIX = df2  # "data/cos_efn128combinedplatesout_conv6a_1e-2_e30.csv"
# OUT_RESUTS = "output/efn128combinedplatesout_conv6a_1e-2_e30"

def load_similarity_matrix(cr_mat):
    # Load matrix in triplet format and reshape
    # cr_mat = pd.read_csv(filename)
    X = cr_mat.pivot(index="Var1", columns="Var2", values="value").reset_index()
    
    # Identify annotations
    Y = cr_mat.groupby("Var1").max().reset_index()
    Y = Y[~Y["Metadata_moa.x"].isna()].sort_values(by="Var1")
    
    # Make sure the matrix is sorted by treatment
    X = X.loc[X.Var1.isin(Y.Var1), ["Var1"] + list(Y.Var1)].sort_values("Var1")
    
    return X,Y

X, Y = load_similarity_matrix(SIM_MATRIX)  # X 加载了数值, Y 加载了treatment等信息，最后变成随机量。
# Y

In [91]:
# Y

In [ ]:
from utils import *

# MOA matching

Y.groupby("Metadata_moa.x")["Var1"].count()  # 找到每一种 MOA 中 Var1：Treatment 的数量

moa_matches = []
Y["Ref_moa"] = Y["Metadata_moa.x"].str.replace('|', '___')  #　potassium channel activator	
# Y['Metadata_moa.x'][63] 

# MOA 是 Metadata_moa.x 的内部结果，如果 Metadata_moa.x 包含多个预测，则 MOA 中包含多个 True
Y["Ref_moa"] = Y["Metadata_moa.x"].str.replace('|', '___')  #　内部包含多项的预测，替换后方便使用正则表达式进行匹配。 'norepinephrine reuptake inhibitor|tricyclic antidepressant'
for k,r in Y.iterrows():
    moas = r["Metadata_moa.x"].split("|")
    # print(moas)
    candidates = []
    for m in moas:
        reg = r'(^|___){}($|___)'.format(m)  
        '''
        正则表达式：
        匹配字符串 m，并确保它要么出现在字符串的开头或结尾，要么被三个下划线分隔。例如，如果 m 是 example，那么生成的正则表达式将是 (^|___)example($|___)，它可以匹配以下情况：
        example 在字符串的开头或结尾。
        example 被 ___ 分隔，如 ___example___。
        '''
        candidates.append(Y["Ref_moa"].str.contains(reg))
        # print('reg', reg, candidates[:20])
    matches = candidates[0]
    for c in candidates:
        # print("22", matches, c)
        matches = matches | c
    moa_matches.append(matches)
    # break

moa_matches = np.asarray(moa_matches)
# plt.imshow(moa_matches)


# # Enrichment analysis

# %% [markdown]
# # 输入
# 相似矩阵 (SIM)：一个表示样本或基因之间相似性的矩阵。
# 匹配数据 (moa_matches)：一个包含匹配信息的数据集。
# 阈值 (threshold)：一个数值参数，用于控制分析的严格程度。
# # 输出
# 富集结果：通常是一个包含富集分析结果的列表或数据框，可能包括显著性值、富集分数等。
# 可视化图表：一些函数可能会生成热图、条形图等用于展示富集结果的图表。

results = {}
SIM = np.asarray(X[Y.Var1])
# print("SIM:", SIM.shape)  # (995, 995)
# print(SIM)

is_query = moa_matches.sum(axis=0) > 1 
#　计算 moa_matches 每列的和，并判断是否大于1，结果存储在布尔数组 is_query 中。 大于1：表示该列中至少有两个或更多的非零值。这意味着在 moa_matches 中，该列有多个匹配项。

for i in range(SIM.shape[0]):
    if is_query[i]: #　如果 is_query 中对应位置为 True, 即大于1，有多个匹配项的情况。才能计算富集分析。
        idx = [x for x in range(SIM.shape[1]) if x != i] #　创建一个索引列表 idx，包含除了当前行 i 之外的所有列索引。除开对角线。
        results[i] = enrichment_analysis(SIM[i,idx], moa_matches[i,idx], 99.) # 确认这两个列表中，匹配情况是否高于随即情况
        # 对 SIM 的第 i 行（去掉第 i 列）和 moa_matches 的第 i 行（去掉第 i 列）进行富集分析，并将结果存储在 results 的第 i 个位置。
        if results[i]["ods_ratio"] is np.nan: # ods_ratio大于1 表明SIM[i,idx]中命中的概率高于在 moa_matches[i, idx] 中的概率
            print(results[i]["V"], i)
# results

# 计算并打印富集分析结果中 ods_ratio 的平均值
# 大于 1 则表明： SIM[i, idx] 中，该事件或特征更为显著或富集

folds = [results[x]["ods_ratio"] for x in results] # 提取所有 ods_ratio
enrichment_top_1 = np.mean(folds)
# print("Average folds of enrichment at top 1%:", enrichment_top_1)


enrichment_results = pd.DataFrame(data=results).T
# enrichment_results

# # Average precision analysis
import numpy as np
import pandas as pd

def precision_at_k(sim_matrix, moa_matches, k):
    """Calculate precision at k for each query"""
    results = {}
    is_query = moa_matches.sum(axis=0) > 1  # Only calculate for queries with multiple positives
    
    for i in range(sim_matrix.shape[0]):
        if is_query[i]:
            ranking = np.argsort(-sim_matrix[i, :])  # Descending order
            top_k_matches = moa_matches[i, ranking[1:k+1]]  # Exclude self, get top k
            pk = np.sum(top_k_matches) / k
            results[i] = {"precision_at_k": pk, "k": k}
    return results

def recall_at_k(sim_matrix, moa_matches, k):
    """Calculate recall at k for each query"""
    results = {}
    is_query = moa_matches.sum(axis=0) > 1
    total_positives = moa_matches.sum(axis=1)
    
    for i in range(sim_matrix.shape[0]):
        if is_query[i] and total_positives[i] > 0:
            ranking = np.argsort(-sim_matrix[i, :])
            top_k_matches = moa_matches[i, ranking[1:k+1]]
            recall = np.sum(top_k_matches) / total_positives[i]
            results[i] = {
                "recall_at_k": recall,
                "baseline_recall": np.mean(moa_matches),  # Random baseline
                "k": k
            }
    return results

def evaluate_model(sim_matrix, moa_matches):
    """Comprehensive evaluation of retrieval performance"""
    # Fixed evaluation points
    evaluation_points = [5, 10, 20, 50, 100]
    evaluation_percents = [1, 3, 5, 10, 20]
    
    # Calculate absolute positions for percentages
    n = sim_matrix.shape[0]
    percent_positions = [max(int(n * p/100), 1) for p in evaluation_percents]
    
    # Store all results
    results = {
        'precision': {},
        'recall': {},
        'metrics': {}
    }
    
    # Calculate precision@k
    for k in evaluation_points:
        prec_k = precision_at_k(sim_matrix, moa_matches, k)
        avg_prec = np.mean([prec_k[q]["precision_at_k"] for q in prec_k])
        results['precision'][f'P@{k}'] = avg_prec
    
    # Calculate recall@k and recall@%
    for pos, percent in zip(percent_positions, evaluation_percents):
        # Precision at percentage
        prec_p = precision_at_k(sim_matrix, moa_matches, pos)
        avg_prec_p = np.mean([prec_p[q]["precision_at_k"] for q in prec_p])
        results['precision'][f'P@{percent}%'] = avg_prec_p
        
        # Recall at percentage
        recall_p = recall_at_k(sim_matrix, moa_matches, pos)
        avg_recall_p = np.mean([recall_p[q]["recall_at_k"] for q in recall_p])
        baseline_p = np.mean([recall_p[q]["baseline_recall"] for q in recall_p])
        results['recall'][f'R@{percent}%'] = {
            'value': avg_recall_p,
            'baseline': baseline_p,
            'improvement': avg_recall_p / baseline_p if baseline_p > 0 else np.nan
        }
    
    # Calculate MAP (Mean Average Precision)
    map_score = calculate_map(sim_matrix, moa_matches)
    results['metrics']['MAP'] = map_score
    
    return results

def calculate_map(sim_matrix, moa_matches):
    """Calculate Mean Average Precision without interpolation"""
    aps = []
    is_query = moa_matches.sum(axis=0) > 1
    total_positives = moa_matches.sum(axis=1)
    
    for i in range(sim_matrix.shape[0]):
        if is_query[i] and total_positives[i] > 0:
            ranking = np.argsort(-sim_matrix[i, :])
            relevant = moa_matches[i, ranking[1:]]  # Exclude self
            
            # Calculate precision at each rank where recall increases
            precisions = []
            true_positives = 0
            for k in range(len(relevant)):
                if relevant[k]:
                    true_positives += 1
                    precisions.append(true_positives / (k + 1))
            
            if precisions:
                ap = np.sum(precisions) / total_positives[i]
                aps.append(ap)
    
    return np.mean(aps) if aps else 0

# Example usage:
results = evaluate_model(SIM, moa_matches)
print("Evaluation Results:")
print("Precision metrics:", results['precision'])
print("Recall metrics:", results['recall'])
print("MAP:", results['metrics']['MAP'])

print("Average folds of enrichment at top 1%:", enrichment_top_1)
# print("Mean Average Precision (MAP): \t", np.mean(average_precision))

/tmp/ipykernel_567401/1900701817.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  candidates.append(Y["Ref_moa"].str.contains(reg))


Evaluation Results:
Precision metrics: {'P@5': 0.03255813953488373, 'P@10': 0.02441860465116279, 'P@20': 0.020348837209302327, 'P@50': 0.013720930232558142, 'P@100': 0.01, 'P@1%': 0.05813953488372093, 'P@3%': 0.05813953488372093, 'P@5%': 0.0377906976744186, 'P@10%': 0.02454780361757106, 'P@20%': 0.020671834625322995}
Recall metrics: {'R@1%': {'value': 0.029069767441860465, 'baseline': 0.02172839506172839, 'improvement': 1.3378699788583512}, 'R@3%': {'value': 0.05813953488372093, 'baseline': 0.02172839506172839, 'improvement': 2.6757399577167025}, 'R@5%': {'value': 0.0755813953488372, 'baseline': 0.02172839506172839, 'improvement': 3.478461945031713}, 'R@10%': {'value': 0.11046511627906977, 'baseline': 0.02172839506172839, 'improvement': 5.083905919661735}, 'R@20%': {'value': 0.18604651162790697, 'baseline': 0.02172839506172839, 'improvement': 8.562367864693448}}
MAP: 0.06817616904687326
Average folds of enrichment at top 1%: 5.116279069767442


: 